In [ ]:
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass


from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.azure import AzureProvider

model = OpenAIChatModel("gpt-4.1-mini", provider=AzureProvider())

agent = Agent(
    model=model,
    instructions=(
        "You are an expert when it comes to writing complete and detailed docstrings "
        "from not very informative tool definitions. "
        "Your task is to improve the tool descriptions by infering their intended use "
        "from the context and from the parameter json schema. "
        "All these tools are used for the AI Knowledge API, which is a backend service "
        "for managing document collections, indexing, and semantic search. "
        "Make sure the docstrings follow NumPy style formatting guidelines. "
        "This is important: Only answer with the new docstring, do not include any other text!"
    ),
)

In [ ]:
import sys

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

from meta_ally.lib.openapi_to_tools import OpenAPIToolsLoader

# Initialize the loader for AI Knowledge API
loader = OpenAPIToolsLoader(
    openapi_url=(
        "https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json"
    )
)

# Load all tools
print("Loading tools from AI Knowledge API OpenAPI spec...")
tools = loader.load_tools()

Loading tools from AI Knowledge API OpenAPI spec...
Created tool: list_collections [GET /collections/]
Created tool: create_collection [POST /collections/]
Created tool: get_collection_configuration [GET /collections/{collection_id}]
Created tool: update_collection [PUT /collections/{collection_id}]
Created tool: delete_collection [DELETE /collections/{collection_id}]
Created tool: index_collection [POST /collections/{collection_id}/index]
Created tool: query [POST /collections/{collection_id}/query]
Created tool: get_fragment_counts [GET /collections/{collection_id}/fragments/counts]
Created tool: get_collection_usage [GET /collections/{collection_id}/usage]
Created tool: list_sources [GET /sources/]
Created tool: create_source [POST /sources/]
Created tool: get_source_configuration [GET /sources/{source_id}]
Created tool: update_source [PUT /sources/{source_id}]
Created tool: delete_source [DELETE /sources/{source_id}]
Created tool: health_check [GET /api/test]
Created tool: check_co

In [9]:
for tool in tools:
    print(f"{tool.tool_def}\n")

ToolDefinition(name='list_collections', parameters_json_schema={'type': 'object', 'properties': {'lastIndexRun': {'type': 'string', 'description': 'Flag to return the last index run of each collection.'}, 'nextIndexRun': {'type': 'string', 'description': 'Flag to return the next scheduled index run of each collection.'}, 'collectionId': {'type': 'string', 'description': "Collection ID(s) to filter by. This is a 'contains' filter - supports multiple values with OR logic."}, 'description': {'type': 'string', 'description': "Description(s) to filter by. This is a 'contains' filter - supports multiple values with OR logic."}, 'public': {'type': 'string', 'description': 'Flag to filter by public collections.'}, 'accessDenied': {'type': 'string', 'description': 'Access denied flag(s) to filter by. Supports multiple values with OR logic.'}, 'businessDepartment': {'type': 'string', 'description': "Business department to filter by. This is a 'contains' filter."}, 'projectNumber': {'type': 'stri

In [ ]:
special_tool = loader.get_tool_by_operation_id("update_source")
if special_tool is not None:
    print(special_tool.tool_def.parameters_json_schema)

{'type': 'object',
 'properties': {'source_id': {'type': 'string', 'description': ''},
  'sourceId': {'type': 'string',
   'maxLength': 255,
   'minLength': 8,
   'pattern': '^[a-zA-Z0-9-_]+$',
   'title': 'Sourceid',
   'description': 'A unique identifier for the data source.'},
  'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'title': 'Description',
   'description': 'Description of the data source.'},
  'properties': {'oneOf': [{'properties': {'prefix': {'items': {'type': 'string'},
       'type': 'array',
       'maxItems': 32,
       'minItems': 0,
       'title': 'Prefix',
       'description': 'A list of prefixes to filter objects in the source.',
       'default': []},
      'suffix': {'items': {'type': 'string'},
       'type': 'array',
       'maxItems': 32,
       'minItems': 0,
       'title': 'Suffix',
       'description': 'A list of suffixes to filter objects in the source.',
       'default': []},
      'prefixIgnore': {'items': {'type': 'string'},


In [ ]:
# Collect improved docstrings and write to markdown file
improved_docstrings = []

for tool in tools:
    tool_def = tool.tool_def
    new_docstring = await agent.run(
        f"Generate a new improved docstring for me:\n\n{tool_def}"
    )
    result = new_docstring.output.strip()

    improved_docstrings.append({
        'name': tool.name,
        'docstring': result
    })

    print(f"Processed tool: {tool.name}")

print(f"\nProcessed {len(improved_docstrings)} tools total.")

Processed tool: list_collections
Processed tool: create_collection
Processed tool: create_collection
Processed tool: get_collection_configuration
Processed tool: get_collection_configuration
Processed tool: update_collection
Processed tool: update_collection
Processed tool: delete_collection
Processed tool: delete_collection
Processed tool: index_collection
Processed tool: index_collection
Processed tool: query
Processed tool: query
Processed tool: get_fragment_counts
Processed tool: get_fragment_counts
Processed tool: get_collection_usage
Processed tool: get_collection_usage
Processed tool: list_sources
Processed tool: list_sources
Processed tool: create_source
Processed tool: create_source
Processed tool: get_source_configuration
Processed tool: get_source_configuration
Processed tool: update_source
Processed tool: update_source
Processed tool: delete_source
Processed tool: delete_source
Processed tool: health_check
Processed tool: health_check
Processed tool: check_connection_s3
Pro

In [ ]:
# Write improved docstrings to markdown file
markdown_content = "# Improved AI Knowledge Tool Docstrings\n\n"
markdown_content += (
    "This document contains improved docstrings for tools generated "
    "from the AI Knowledge API OpenAPI specification.\n\n"
)
markdown_content += "---\n\n"

for item in improved_docstrings:
    markdown_content += f"## {item['name']}\n\n"
    markdown_content += f"```\n{item['docstring']}\n```\n\n"
    markdown_content += "---\n\n"

# Write to file
with open("ai_knowledge_improved_tool_descriptions.md", "w", encoding="utf-8") as f:
    f.write(markdown_content)

print(
    "✅ Markdown file 'ai_knowledge_improved_tool_descriptions.md' "
    "has been created successfully!"
)
print(f"📄 Contains improved docstrings for {len(improved_docstrings)} tools.")

✅ Markdown file 'ai_knowledge_improved_tool_descriptions.md' has been created successfully!
📄 Contains improved docstrings for 35 tools.
